<a href="https://colab.research.google.com/github/mathu3004/Pearl_Path/blob/Personalized_Itinerary_Generator_Based_Radius/Model_Training_for_Travel_Planner_Based_Radius.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, MultiLabelBinarizer
from geopy.geocoders import Nominatim
from urllib.parse import urlparse

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Restaurants Data Preprocessed
# Load the merged CSV file
file_path = '/content/drive/My Drive/Colombo/FinalPreprocessedMergedRestaurants.csv'
df = pd.read_csv(file_path)

# 1. Address Preprocessing
# Geocode to latitude and longitude (if needed)
geolocator = Nominatim(user_agent="geoapiExercises")
def get_coordinates(address):
    try:
        location = geolocator.geocode(address)
        return location.latitude, location.longitude
    except:
        return None, None

# Uncomment if geocoding is needed
#df['latitude'], df['longitude'] = zip(*df['address'].apply(get_coordinates))

# 2. Category Encoding
label_encoder = LabelEncoder()
df['category_encoded'] = label_encoder.fit_transform(df['category'])

# 5. Multi-Label Columns Preprocessing
def process_list_columns(column):
    return column.apply(lambda x: [item.strip() for item in str(x).split(',')] if pd.notnull(x) else [])

# Step 1: Create a mapping dictionary for city grouping
city_mapping = {
    # Map to Nuwara Eliya
    'Kotmale': 'Nuwara Eliya',
    'Nanu Oya': 'Nuwara Eliya',
    'Ramboda': 'Nuwara Eliya',
    'Talawakele': 'Nuwara Eliya',
    'Ambewela': 'Nuwara Eliya',

    # Map to Ella
    'Haputale': 'Ella',
    'Bandarawela': 'Ella',
    'Kithalella': 'Ella',
    'Wellawaya': 'Ella',

    # Map to Kandy
    'Peradeniya': 'Kandy',
    'Gampola': 'Kandy',
    'Kadugannawa': 'Kandy',
    'Tennekumbura': 'Kandy',

    # Map to Colombo
    'Kadawata': 'Colombo',
    'Maharagama': 'Colombo',
    'Wattala': 'Colombo',
    'Kaduwela': 'Colombo',
    'Rajagiriya': 'Colombo',
    'Battaramulla': 'Colombo',
    'Dehiwala-Mount Lavinia': 'Colombo',
    'Nugegoda': 'Colombo',
    'Mount Lavinia': 'Colombo',
    'Uswetakeiyawa': 'Colombo',
    'Boralesgamuwa': 'Colombo',
    'Malabe': 'Colombo',
    'Thalawathugoda': 'Colombo',
    'Dehiwala': 'Colombo',
    'Kelaniya': 'Colombo',
    'Kiribathgoda': 'Colombo',
    'Panadura': 'Colombo'
}

# Apply the city mapping to create a permanent city column
df['city'] = df['addressobj_city'].replace(city_mapping)

# Ensure all other cities are labeled correctly
df['city'] = df['city'].apply(lambda x: x if x in ['Nuwara Eliya', 'Ella', 'Kandy', 'Colombo'] else 'Other')

# 3. One-Hot Encoding to create only four main city columns
city_dummies = pd.get_dummies(df['city'], prefix='city')[['city_Colombo', 'city_Ella', 'city_Kandy', 'city_Nuwara Eliya']].astype(int)
df = pd.concat([df, city_dummies], axis=1)

# drop addressobj_city column
df.drop(columns=['addressobj_city'], inplace=True)

df['cuisines'] = process_list_columns(df['cuisines'])
df['features'] = process_list_columns(df['features'])
df['dietaryrestrictions'] = process_list_columns(df['dietaryrestrictions'])
df['mealtypes'] = process_list_columns(df['mealtypes'])

# One-Hot Encoding for multi-label data
mlb = MultiLabelBinarizer()
# Create DataFrames for one-hot encoded features and add prefix to column names
cuisine_df = pd.DataFrame(mlb.fit_transform(df['cuisines']), columns=mlb.classes_, index=df.index)
cuisine_df = cuisine_df.add_prefix('cuisine_')  # Add prefix here
df = df.join(cuisine_df)

feature_df = pd.DataFrame(mlb.fit_transform(df['features']), columns=mlb.classes_, index=df.index)
feature_df = feature_df.add_prefix('feature_')  # Add prefix here
df = df.join(feature_df)

dietary_df = pd.DataFrame(mlb.fit_transform(df['dietaryrestrictions']), columns=mlb.classes_, index=df.index)
dietary_df = dietary_df.add_prefix('dietary_')  # Add prefix here
df = df.join(dietary_df)

mealtype_df = pd.DataFrame(mlb.fit_transform(df['mealtypes']), columns=mlb.classes_, index=df.index)
mealtype_df = mealtype_df.add_prefix('mealtype_')  # Add prefix here
df = df.join(mealtype_df)

# Save the final cleaned data to a new file
final_processed_file_path = "/content/drive/My Drive/DataPre/Restaurants/LastPreprocessedMergedRestaurants.csv"
df.to_csv(final_processed_file_path, index=False)

In [ ]:
# User Data Preprocessing
# Load the uploaded file
file_path = "/content/drive/My Drive/DataPre/User/UserInputs.csv"
df = pd.read_csv(file_path)

# Generate synthetic 'maximum_distance' column with values ranging from 10km to 50km
df['maximum_distance'] = np.random.randint(10, 51, size=len(df))

# Define a dictionary to map old column names to new names
column_mapping = {
    "name": "name",
    "how_many_people_are_traveling": "peoplecount",
    "destination": "destination",
    "number_of_days": "numberofdays",
    "what_is_your_age_category": "agecategory",
    "accomadation_type": "accomadation_type",
    "budget_per_day": "budget_per_day",
    "food_preference": "food_preference",
    "cuisine_preference": "cuisine_preference",
    "activities_preference": "activities_preference",
    "time_preference_activities": "time_preference_activities",
    "which_transportation_modes_are_you_preferred_for_traveling_within_the_destination": "transportation_mode",
    "children_or_pets": "children_or_pets",
    "maximum_distance": "maximum_distance"
}

# Select and rename columns in one step using the dictionary
df = df.rename(columns=column_mapping)[column_mapping.values()]

# Dropping unnecessary columns if they exist in the dataset
columns_to_drop = [
    "do_you_wish_to_have_a_trip_itinerary_generator_website_for_your_future_trips",
    "is_there_anything_else_you_would_like_us_to_know_about_your_travel_preferences"
]
df = df.drop(columns=columns_to_drop, errors='ignore')

# Convert "budget_per_day" ranges into numerical values (average of range)
def convert_budget_to_numeric(budget):
    budget = str(budget).replace('Rs.', '').strip()  # Remove 'Rs.' prefix
    match = re.findall(r'\d+', budget.replace(',', ''))
    if len(match) == 2:
        return (int(match[0]) + int(match[1])) / 2
    elif len(match) == 1:
        return int(match[0])
    else:
        return None

df['budget_per_day'] = df['budget_per_day'].apply(convert_budget_to_numeric)

# Reapplying preprocessing with multi-label splitting and one-hot encoding for specific columns

from sklearn.preprocessing import MultiLabelBinarizer

# Define multi-label columns for special encoding
multi_label_columns = [
    "cuisine_preference",
    "activities_preference",
    "time_preference_activities",
    "transportation_mode",
    "peoplecount",
    "destination",
    "agecategory",
    "accomadation_type",
    "food_preference",
    "children_or_pets"
]

# Split multi-label columns into lists
for column in multi_label_columns:
    df[column] = df[column].apply(lambda x: [item.strip() for item in str(x).split(',')])

# Apply MultiLabelBinarizer to each multi-label column
mlb = MultiLabelBinarizer()

multi_label_encoded_data = pd.DataFrame()

for column in multi_label_columns:
    # One-hot encode the multi-label columns
    encoded = pd.DataFrame(mlb.fit_transform(df[column]),
                           columns=[f"{column}_{cls}" for cls in mlb.classes_],
                           index=df.index)
    multi_label_encoded_data = pd.concat([multi_label_encoded_data, encoded], axis=1)

# Drop the original multi-label columns from filtered data
df = df.drop(columns=multi_label_columns)

# Combine the multi-label encoded columns with the rest of the encoded data
df = pd.concat([df, multi_label_encoded_data], axis=1)

# Save the updated file back to Google Drive
file_path = "/content/drive/My Drive/DataPre/User/PreprocessedUserInputs.csv"
df.to_csv(file_path, index=False)

print("File successfully saved!")

File successfully saved!


In [ ]:
#Attraction Preprocessing
import pandas as pd
import os
import glob
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# List of folder paths where the CSVs are stored
folders = [
    "/content/drive/My Drive/DataPre/Attractions"
]

# Initialize an empty list to store DataFrames
dataframes = []

# Iterate over each folder and read only CSV files that start with "Attractions"
for folder in folders:
    csv_files = glob.glob(os.path.join(folder, "Attractions*.csv"))  # Corrected file pattern

    for file in csv_files:
        df = pd.read_csv(file)  # Read CSV
        dataframes.append(df)   # Append DataFrame to the list

# Concatenate all DataFrames into a single one
merged_df = pd.concat(dataframes, ignore_index=True)

# Save the merged DataFrame as a CSV
merged_file_path = "/content/drive/My Drive/DataPre/Attractions/MergedAttractions.csv"
merged_df.to_csv(merged_file_path, index=False)

# Load the uploaded file
file_path = "/content/drive/My Drive/DataPre/Attractions/MergedAttractions.csv"
df = pd.read_csv(file_path)

def clean_and_process_df(df):
    # Standardize column names
    df.columns = [col.strip().lower().replace(' ', '').replace('/', '') for col in df.columns]

    # Define the required columns
    required_columns = [
        'name', 'address', 'addressobjcity', 'category',
        'latitude', 'longitude', 'rankingposition',
        'rating', 'offergrouplowestprice', 'weburl'
    ]

    # Include subcategories and subtype columns dynamically
    subcategory_columns = [f'subcategories{i}' for i in range(10)]
    subtype_columns = [f'subtype{i}' for i in range(46)]

    all_required_columns = required_columns + subcategory_columns + subtype_columns

    # Drop columns that are not in the required columns list and create a copy
    df = df[[col for col in df.columns if col in all_required_columns]].copy()

    # Combine subcategories and subtype columns into separate columns
    subcategory_columns = [col for col in df.columns if col.startswith('subcategories')]
    subtype_columns = [col for col in df.columns if col.startswith('subtype')]

    # Concatenate subcategories into 'SubCategory' column, excluding NaN values
    df['SubCategory'] = df[subcategory_columns].apply(
        lambda row: ', '.join([str(x) for x in row if pd.notna(x) and x and x.lower() != 'none']),
        axis=1
    )

    # Concatenate subtypes into 'Subtype' column, excluding NaN values
    df['Subtype'] = df[subtype_columns].apply(
        lambda row: ', '.join([str(x) for x in row if pd.notna(x) and x and x.lower() != 'none']),
        axis=1
    )

    # Drop the original subcategory and subtype columns to clean up
    df.drop(columns=subcategory_columns + subtype_columns, inplace=True)

    # Rename columns as specified
    df = df.rename(columns={
        'name': 'Name',
        'address': 'Address',
        'addressobjcity': 'City',
        'category': 'Category',
        'latitude': 'Latitude',
        'longitude': 'Longitude',
        'rankingposition': 'Ranking_Position',
        'rating': 'Rating',
        'offergrouplowestprice': 'Lowest Price',
        'weburl': 'Web URL'
    })

    # Fill missing values using .loc to avoid SettingWithCopyWarning
    # Convert Ranking_Position to numeric, fill missing with the mode
    df['Ranking_Position'] = pd.to_numeric(df['Ranking_Position'], errors='coerce')
    mode_value = df['Ranking_Position'].mode()[0] if not df['Ranking_Position'].mode().empty else 0
    df['Ranking_Position'] = df['Ranking_Position'].fillna(mode_value)
    #drop if it no longitude no latitude
    df = df[~df['Latitude'].isin([np.NaN]) & ~df['Longitude'].isin([np.NaN]) & ~df['City'].isin([np.NaN])]

    # Convert the 'Lowest Price' to LKR if it is in USD
    def convert_to_lkr(price):
        if isinstance(price, str) and price.startswith('$'):
            try:
                usd_value = float(price.replace('$', '').strip())
                lkr_value = usd_value * 320  # Convert USD to LKR
                return round(lkr_value, 2)   # Keep as float with 2 decimal points
            except ValueError:
                return np.nan  # Return NaN for invalid prices
        try:
            # If the price is already in LKR, convert it to a float
            return float(price)
        except ValueError:
            return np.nan  # Return NaN if conversion fails

    # Apply the conversion to the 'Lowest Price' column
    df['Lowest Price'] = df['Lowest Price'].apply(convert_to_lkr)

    # Calculate the mode of the 'Lowest Price' column
    mode_price = df['Lowest Price'].mode()[0] if not df['Lowest Price'].mode().empty else 'No price mentioned'

    # Fill missing values with the mode value
    df['Lowest Price'] = df['Lowest Price'].fillna(mode_price)

    # Generate a random duration between 1 to 4 hours
    df['Duration'] = np.random.choice([1, 2, 3, 4], size=len(df))

    # Define city mappings based on the provided criteria
    city_mapping = {
        'Peradeniya': 'Kandy',
        'Gampola': 'Kandy',
        'Pussellawa': 'Kandy',
        'Heeloya': 'Kandy',
        'Gurudeniya': 'Kandy',
        'Pilimathalawa': 'Kandy',
        'Murutalawa': 'Kandy',

        'Badulla': 'Ella',
        'Nanu Oya': 'Ella',
        'Demodara': 'Ella',
        'Kalupahana': 'Ella',
        'Bandarawela': 'Ella',
        'Ambagollapathana': 'Ella',

        'Hatton': 'Nuwara Eliya',

        'Negombo': 'Colombo',
        'Wattala': 'Colombo',
        'Katunayaka': 'Colombo',
        'Kaduwela': 'Colombo',
        'Katunayake': 'Colombo'
    }

    # Apply the mapping to the 'City' column
    df['City'] = df['City'].apply(lambda x: city_mapping.get(x, x))

    return df

# Clean and process the merged dataset
df = clean_and_process_df(merged_df)

from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
import pandas as pd

# Multi-label encoding for 'City', 'SubCategory', and 'Subtype' columns
def multi_label_encoding(df, columns):
    for column in columns:
        mlb = MultiLabelBinarizer()
        # Split by comma, strip whitespace, and handle empty strings
        encoded_data = mlb.fit_transform(df[column].apply(lambda x: [item.strip() for item in str(x).split(',') if item.strip()]))
        encoded_df = pd.DataFrame(encoded_data, columns=[f"{column}_{cls}" for cls in mlb.classes_])
        df = pd.concat([df, encoded_df], axis=1)
    return df

# One-hot encoding for 'Category' column
def category_encoding(df, column):
    ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

    # Apply OneHotEncoder
    encoded_data = ohe.fit_transform(df[[column]])

    # Generate column names dynamically based on categories
    category_columns = [f"{column}_{cls}" for cls in ohe.categories_[0]]

    # Ensure the shape of data matches the expected columns
    encoded_df = pd.DataFrame(encoded_data, columns=category_columns, index=df.index)

    # Concatenate the encoded data with the original DataFrame
    df = pd.concat([df, encoded_df], axis=1)

    return df

# Apply multi-label encoding
multi_label_columns = ['City', 'SubCategory', 'Subtype']
df = multi_label_encoding(df, multi_label_columns)

# Apply category encoding
df = category_encoding(df, 'Category')

# Save the cleaned DataFrame to a new CSV file
output_file = '/content/drive/My Drive/DataPre/Attractions/PreprocessedMergedAttractions.csv'
df.to_csv(output_file, index=False)

print(f"Processing complete. Cleaned data saved to '{output_file}'")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing complete. Cleaned data saved to '/content/drive/My Drive/DataPre/Attractions/PreprocessedMergedAttractions.csv'
